In [10]:
!pip install langchain==0.1.7 langchain-community==0.0.24
!pip install chromadb==0.4.14
!pip install sentence-transformers transformers


INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install langchain-community==0.0.24 and langchain==0.1.7 because these package versions have conflicting dependencies.

The conflict is caused by:
    langchain 0.1.7 depends on langsmith<0.1 and >=0.0.83
    langchain-community 0.0.24 depends on langsmith<0.2.0 and >=0.1.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline
import torch
import chromadb

"imports OK"


'imports OK'

In [15]:
!curl -L \
  https://raw.githubusercontent.com/ValLovaton/rag_wikipedia-lab/valeria-task2/data/wiki_corpus.csv \
  -o wiki_corpus.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29642  100 29642    0     0  63393      0 --:--:-- --:--:-- --:--:-- 63473


In [16]:
import pandas as pd
df = pd.read_csv("wiki_corpus.csv")
df.head()


,id,title,text
0,0,Federated_learning,Federated learning (also known as collaborativ...
1,1,Federated_learning,smartphones and IoT devices) compared to distr...
2,2,Federated_learning,federated learning In the decentralized federa...
3,3,Federated_learning,"according to the server inputs, a machine lear..."
4,4,Federated_learning,and identically distributed samples across loc...


In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# 1. Crear función de embeddings
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Crear el vector store PERSISTENTE
db = Chroma(
    collection_name="wiki_ai",
    embedding_function=embedding_function,
    persist_directory="/content/chroma_store"
)

# 3. Insertar todos los textos (solo si la colección está vacía)
db.add_texts(
    texts=df["text"].tolist(),
    metadatas=[{"title": t} for t in df["title"]],
    ids=df["id"].astype(str).tolist()
)

# 4. Guardar en disco
db.persist()

"vector store persistido"


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


'vector store persistido'

In [18]:
!zip -r chroma_store.zip /content/chroma_store


  adding: content/chroma_store/ (stored 0%)
  adding: content/chroma_store/chroma.sqlite3 (deflated 69%)
  adding: content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/ (stored 0%)
  adding: content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/header.bin (deflated 61%)
  adding: content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/data_level0.bin (deflated 7%)
  adding: content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/link_lists.bin (stored 0%)
  adding: content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/length.bin (deflated 56%)


In [19]:
!curl -L \
  https://raw.githubusercontent.com/ValLovaton/rag_wikipedia-lab/sebas-task2/data/chroma_store.zip \
  -o chroma_store.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1660k  100 1660k    0     0  1566k      0  0:00:01  0:00:01 --:--:-- 1567k


In [20]:
!unzip -o chroma_store.zip -d /content/


Archive:  chroma_store.zip
   creating: /content/content/chroma_store/
  inflating: /content/content/chroma_store/chroma.sqlite3  
   creating: /content/content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/
  inflating: /content/content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/header.bin  
  inflating: /content/content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/data_level0.bin  
 extracting: /content/content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/link_lists.bin  
  inflating: /content/content/chroma_store/a9faf682-079f-486a-af91-eea432ec5e6a/length.bin  


In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# 1. Embeddings (mismo modelo)
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Cargar la colección persistente
db = Chroma(
    collection_name="wiki_ai",
    embedding_function=embedding_function,
    persist_directory="/content/chroma_store"
)

retriever = db.as_retriever(search_kwargs={"k": 4})

"retriever listo"


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


'retriever listo'

In [22]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

llm_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=512,
    device=0 if torch.cuda.is_available() else -1
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

"llm listo"


Device set to use cpu


'llm listo'

In [23]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

"retrievalQA listo"


'retrievalQA listo'

In [24]:
qa.invoke({"query": "Explain the main challenges of federated learning in healthcare."})


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
Token indices sequence length is longer than the specified maximum sequence length for this model (1720 > 512). Running this sequence through the model will result in indexing errors


{'query': 'Explain the main challenges of federated learning in healthcare.',
 'result': "Heterogeneity between the different local datasets: each node may have some bias with respect to the general population, and the size of the datasets may vary significantly; Temporal heterogeneity: each local dataset's distribution may vary with time; Interoperability of each node's dataset is a prerequisite; Each node's dataset may require regular curations; Hiding training data might allow attackers to inject backdoors into the global model; Lack of access to global training data makes it harder to identify unwanted biases entering the training e.g. age, gender, sexual orientation; Partial or total loss of model updates due to node failures affecting the global model; Lack of annotations or labels on the client side. Heterogeneity between processing platforms"}

In [25]:
summary_query = "Generate a detailed 400-500 word summary explaining what federated learning is, how it works, and its key challenges, using the retrieved evidence."

summary = qa.invoke({"query": summary_query})["result"]
summary


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


'Federated learning is a machine learning technique in a setting where multiple entities (often called clients) collaboratively train a model while keeping their data decentralized, rather than centrally stored. A defining characteristic of federated learning is data heterogeneity. Because client data is decentralized, data samples held by each client may not be independently and identically distributed. Federated learning is generally concerned with and motivated by issues such as data privacy, data minimization, and data access rights. Its applications involve a variety of research areas including defence, telecommunications, the Internet of things, and pharmaceuticals.'

In [26]:
with open("rag_summary.md", "w") as f:
    f.write(summary)


In [27]:
import json

queries = [
    "What are the main challenges of federated learning?",
    "Explain how federated learning differs from traditional centralized training.",
    "Why is data heterogeneity a problem in federated learning?",
    "How does federated learning address privacy concerns?",
]

examples = {}

for q in queries:
    result = qa.invoke({"query": q})["result"]
    examples[q] = result

# Guardar como JSON
with open("retrieval_examples.json", "w") as f:
    json.dump(examples, f, indent=4)

"json generado"


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


'json generado'